#1. Imports

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import librosa
import librosa.display
import os
import pandas as pd
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

# Diretórios contendo as músicas originais e seus covers
dir_originals = '/content/drive/MyDrive/Trabalhos UFMG/MIR/Beatles dataset (MIR)/beatles_songs_stft'
dir_covers = '/content/drive/MyDrive/Trabalhos UFMG/MIR/Beatles dataset (MIR)/beatles_covers_stft'


#2. DTW

##2.0 Verificando as dimensões dos arquivos
Importante para a aplicação do DTW (colunas devem ter a mesma dimensão, mas o tempo pode variar).

In [ ]:
for f, arr in originals.items():
    print(f, arr.shape)

for f, arr in covers.items():
    print(f, arr.shape)


07 The Beatles - This Boy.flac.npy (1025, 1276)
07 The Beatles - She Said She Said.flac.npy (1025, 1284)
08 The Beatles - Roll Over Beethoven.flac.npy (1025, 1277)
08 The Beatles - Revolution.flac.npy (1025, 1276)
08 The Beatles - Love Me Do.flac.npy (1025, 1277)
08 The Beatles - Komm, Gib Mir Deine Hand.flac.npy (1025, 1278)
08 The Beatles - Good Day Sunshine.flac.npy (1025, 1280)
08 The Beatles - Eight Days A Week.flac.npy (1025, 1276)
08 The Beatles - Any Time At All.flac.npy (1025, 1284)
08 The Beatles - Act Naturally.flac.npy (1025, 1280)
07 The Beatles - Ticket To Ride.flac.npy (1025, 1284)
09 The Beatles - Girl.flac.npy (1025, 1284)
09 The Beatles - And Your Bird Can Sing.flac.npy (1025, 1288)
08 The Beatles - Within You, Without You.flac.npy (1025, 1213)
08 The Beatles - What Goes On.flac.npy (1025, 1276)
08 The Beatles - Strawberry Fields Forever.flac.npy (1025, 1269)
09 The Beatles - Penny Lane.flac.npy (1025, 1266)
09 The Beatles - P.S  I Love You.flac.npy (1025, 1276)
09 Th

##2.1 Usando STFTs
Ideia menos inteligente, pela ordem de complexidade elevada da função :)

In [ ]:
def load_and_fix_stft(filepath):
    """
    Carrega um arquivo .npy que contém STFT em formato (freq, time) ou (time, freq).
    Queremos padronizar para (time, 1025).
    """
    arr = np.load(filepath)

    # Se a primeira dimensão for 1025, significa que provavelmente está no formato (1025, frames).
    # Transpomos para ficar (frames, 1025).
    if arr.shape[0] == 1025:
        arr = arr.T  # Agora fica (frames, 1025)

    # Se mesmo assim a segunda dimensão não for 1025, avisamos e ignoramos.
    if arr.shape[1] != 1025:
        print(f"[Aviso] O arquivo '{filepath}' tem shape {arr.shape}, que não é (X, 1025). Será ignorado.")
        return None

    return arr

# Carrega e padroniza os STFTs das músicas originais
originals = {}
for f in os.listdir(dir_originals):
    if f.endswith('.npy'):
        filepath = os.path.join(dir_originals, f)
        arr = load_and_fix_stft(filepath)
        if arr is not None:
            originals[f] = arr

# Carrega e padroniza os STFTs dos covers
covers = {}
for f in os.listdir(dir_covers):
    if f.endswith('.npy'):
        filepath = os.path.join(dir_covers, f)
        arr = load_and_fix_stft(filepath)
        if arr is not None:
            covers[f] = arr

# Se ambos os dicionários estiverem preenchidos, calculamos o DTW
results = []
for orig_name, orig_stft in originals.items():
    for cover_name, cover_stft in covers.items():
        # Calcula DTW entre as matrizes (time, 1025) de cada música
        distance, _ = fastdtw(orig_stft, cover_stft, dist=euclidean)
        results.append({
            "Original": orig_name,
            "Cover": cover_name,
            "DTW_Distance": distance
        })

# Cria um DataFrame para exibir resultados
df_results = pd.DataFrame(results)

# Exibe (imprime) o DataFrame
print(df_results)

# Se você usar a ferramenta ace_tools:
#import ace_tools as tools
#tools.display_dataframe_to_user(name="Distâncias DTW entre Músicas", dataframe=df_results)


KeyboardInterrupt: 

##2.2 Usando Cromas
Diminui a complexidade em 1000x.

In [ ]:
def stft_to_chroma(filepath, sr=44100, hop_length=512, n_fft=1024):
    """
    Lê um arquivo .npy contendo STFT e converte para cromagrama usando librosa.
    Formato de entrada esperado: (freq, tempo) ou (tempo, freq).
    Saída: cromagrama no formato (frames, 12).
    """
    # Carrega o STFT
    stft_data = np.load(filepath)  # Pode ser (freq, time) ou (time, freq)

    # Se a primeira dimensão for menor que a segunda, é provável que esteja (time, freq).
    # Precisamos de (freq, time) para passar a 'S=' no chroma_stft.
    if stft_data.shape[0] < stft_data.shape[1]:
        stft_data = stft_data.T  # Agora fica (freq, time)

    # Converte STFT em Cromagrama (12 bins)
    # 'S=' deve ser um espectrograma de magnitude não negativa.
    # Se você salvou stft_data já como magnitude (np.abs(STFT)), isso funciona diretamente.
    chroma = librosa.feature.chroma_stft(
        S=stft_data,   # passa o STFT (freq x frames)
        sr=sr,
        hop_length=hop_length,
        n_fft=n_fft,
        n_chroma=12
    )
    # chroma terá shape (12, frames)

    # Transpõe para (frames, 12), padrão que usaremos no DTW
    chroma = chroma.T  # (frames, 12)
    return chroma

# Carrega dicionários de cromagramas, convertendo "on the fly" a partir dos arquivos STFT
originals = {}
for f in os.listdir(dir_originals):
    if f.endswith('.npy'):
        filepath = os.path.join(dir_originals, f)
        c = stft_to_chroma(filepath)
        originals[f] = c

covers = {}
for f in os.listdir(dir_covers):
    if f.endswith('.npy'):
        filepath = os.path.join(dir_covers, f)
        c = stft_to_chroma(filepath)
        covers[f] = c

# DTW entre cada música original e cada cover (agora usando cromas!)
results = []
for orig_name, orig_chroma in originals.items():
    for cover_name, cover_chroma in covers.items():
        distance, _ = fastdtw(orig_chroma, cover_chroma, dist=euclidean)
        results.append({
            "Original": orig_name,
            "Cover": cover_name,
            "DTW_Distance": distance
        })

df_results = pd.DataFrame(results)
print(df_results)


/usr/local/lib/python3.11/dist-packages/librosa/core/pitch.py:101: UserWarning: Trying to estimate tuning from empty frequency set.
  return pitch_tuning(


                                 Original  \
0      07 The Beatles - This Boy.flac.npy   
1      07 The Beatles - This Boy.flac.npy   
2      07 The Beatles - This Boy.flac.npy   
3      07 The Beatles - This Boy.flac.npy   
4      07 The Beatles - This Boy.flac.npy   
...                                   ...   
47952    04 The Beatles - Chains.flac.npy   
47953    04 The Beatles - Chains.flac.npy   
47954    04 The Beatles - Chains.flac.npy   
47955    04 The Beatles - Chains.flac.npy   
47956    04 The Beatles - Chains.flac.npy   

                                                   Cover  DTW_Distance  
0      Please Mister Postman - The Beatles cover [QCT...    168.165103  
1      Robert Gordon - Run For Your Life (The Beatles...    244.102269  
2      heather nova we can work it out [VUunSZkaVgU]....    177.162213  
3        Tony Williams - Blackbird [Mirjk3qXUxg].mp4.npy    189.564241  
4      Los Lobos - Tomorrow Never Knows (A Beatles So...    155.475902  
...                  

In [ ]:
df_results.head(10)

,Original,Cover,DTW_Distance
0,07 The Beatles - This Boy.flac.npy,Please Mister Postman - The Beatles cover [QCT...,168.165103
1,07 The Beatles - This Boy.flac.npy,Robert Gordon - Run For Your Life (The Beatles...,244.102269
2,07 The Beatles - This Boy.flac.npy,heather nova we can work it out [VUunSZkaVgU]....,177.162213
3,07 The Beatles - This Boy.flac.npy,Tony Williams - Blackbird [Mirjk3qXUxg].mp4.npy,189.564241
4,07 The Beatles - This Boy.flac.npy,Los Lobos - Tomorrow Never Knows (A Beatles So...,155.475902
5,07 The Beatles - This Boy.flac.npy,Humble Pie-Drive My Car (The Beatles cover) [z...,203.131866
6,07 The Beatles - This Boy.flac.npy,Bette Midler In My Life Official Video [x6052d...,214.577019
7,07 The Beatles - This Boy.flac.npy,Judy Collins - When I'm Sixty-Four [yRUUWyXBfC...,157.349484
8,07 The Beatles - This Boy.flac.npy,Ringo Starr - Don't Pass Me By (live 2005) HQ ...,199.518330
9,07 The Beatles - This Boy.flac.npy,And Your Bird Can Sing by Jim Reid (Beatles Co...,184.659368


###2.2.1. Salvando em json

In [ ]:
import json

# convertendo para json
data = df_results.to_dict(orient='records')

with open('df_results.json', 'w') as f:
    json.dump(data, f, indent=4)

In [ ]:
# lendo do json
with open('df_results.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame(data)

print(df.head())


                             Original  \
0  07 The Beatles - This Boy.flac.npy   
1  07 The Beatles - This Boy.flac.npy   
2  07 The Beatles - This Boy.flac.npy   
3  07 The Beatles - This Boy.flac.npy   
4  07 The Beatles - This Boy.flac.npy   

                                               Cover  DTW_Distance  
0  Please Mister Postman - The Beatles cover [QCT...    168.165103  
1  Robert Gordon - Run For Your Life (The Beatles...    244.102269  
2  heather nova we can work it out [VUunSZkaVgU]....    177.162213  
3    Tony Williams - Blackbird [Mirjk3qXUxg].mp4.npy    189.564241  
4  Los Lobos - Tomorrow Never Knows (A Beatles So...    155.475902  


In [ ]:
df

,Original,Cover,DTW_Distance
0,07 The Beatles - This Boy.flac.npy,Please Mister Postman - The Beatles cover [QCT...,168.165103
1,07 The Beatles - This Boy.flac.npy,Robert Gordon - Run For Your Life (The Beatles...,244.102269
2,07 The Beatles - This Boy.flac.npy,heather nova we can work it out [VUunSZkaVgU]....,177.162213
3,07 The Beatles - This Boy.flac.npy,Tony Williams - Blackbird [Mirjk3qXUxg].mp4.npy,189.564241
4,07 The Beatles - This Boy.flac.npy,Los Lobos - Tomorrow Never Knows (A Beatles So...,155.475902
...,...,...,...
47952,04 The Beatles - Chains.flac.npy,I'm Down - Aerosmith (Lyrics) [npG_SQaohro].mp...,227.854655
47953,04 The Beatles - Chains.flac.npy,Eight Days a Week - The Runaways [0KOO0Gg1dJg]...,173.709501
47954,04 The Beatles - Chains.flac.npy,Badfinger - Come and get it 1970 [Bk57K4OGrAg]...,159.615348
47955,04 The Beatles - Chains.flac.npy,Tenacious D - MSG And Your Bird Can Sing (Beat...,163.084043
